In [19]:
# Robust CSV loader: tries primary path, catches PermissionError, and falls back to known dataset files
import numpy as np
from pathlib import Path
import pandas as pd
import os

def robust_read_csv(primary_folder, primary_name='Air_Quality.csv'):
    """Try to read a primary CSV and fall back to common alternatives.
    Returns a pandas.DataFrame or raises FileNotFoundError if nothing readable is found.
    """
    primary = Path(primary_folder) / primary_name
    print('Checking primary:', primary)

    if primary.exists():
        try:
            df = pd.read_csv(primary)
            print(f'Loaded primary file: {primary}')
            return df
        except PermissionError as pe:
            print(f'PermissionError reading {primary}:', pe)
        except Exception as e:
            print(f'Error reading {primary}: {type(e).__name__}:', e)
    else:
        print(f'Primary file not found: {primary}')

    # Candidate fallbacks (repo-relative and common filenames)
    candidates = [
        Path.cwd() / 'data' / 'AirQualityUCI.csv',
        Path(primary_folder) / 'AirQualityUCI.csv',
        Path.cwd() / 'data' / 'Air_Quality.csv',
        Path(primary_folder) / 'Air_Quality.csv',
    ]

    for c in candidates:
        try:
            if c.exists():
                print('Trying fallback:', c)
                # The original UCI file uses semicolons
                if 'AirQualityUCI' in c.name:
                    df = pd.read_csv(c, sep=';')
                else:
                    df = pd.read_csv(c)
                print('Loaded fallback:', c)
                return df
        except PermissionError as pe:
            print(f'PermissionError reading fallback {c}:', pe)
        except Exception as e:
            print(f'Failed to load fallback {c}: {type(e).__name__}:', e)

    raise FileNotFoundError(f'No readable CSV found. Checked primary: {primary} and fallbacks: {candidates}')

# Use the robust loader
folder_path = r'C:\Users\folij077\OneDrive - GUSCanada\SCHOOL\Fall 2025\Agile Software Development\CPSC620-air-quality-dashboard\data'
try:
    df = robust_read_csv(folder_path)
    print('Data loaded, shape=', getattr(df, 'shape', None))
except Exception as e:
    print('Final error while loading CSV:', type(e).__name__, e)

Checking primary: C:\Users\folij077\OneDrive - GUSCanada\SCHOOL\Fall 2025\Agile Software Development\CPSC620-air-quality-dashboard\data\Air_Quality.csv
Primary file not found: C:\Users\folij077\OneDrive - GUSCanada\SCHOOL\Fall 2025\Agile Software Development\CPSC620-air-quality-dashboard\data\Air_Quality.csv
Trying fallback: C:\Users\folij077\OneDrive - GUSCanada\SCHOOL\Fall 2025\Agile Software Development\CPSC620-air-quality-dashboard\data\AirQualityUCI.csv
Loaded fallback: C:\Users\folij077\OneDrive - GUSCanada\SCHOOL\Fall 2025\Agile Software Development\CPSC620-air-quality-dashboard\data\AirQualityUCI.csv
Data loaded, shape= (9471, 17)


In [22]:
def clean_data(df):
    """
    Clean the air quality dataset by handling missing values and data types.
    
    The source dataset encodes missing values as -200. These are converted to NaN.
    
    Args:
        df (pd.DataFrame): Raw dataset
        
    Returns:
        pd.DataFrame: Cleaned dataset
    """
    if df is None:
        return None
    
    MISSING_VALUE_CODE = -200
    # Create a copy to avoid modifying the original
    df_clean = df.copy()
    
    # Convert date and time columns
    df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%d/%m/%Y', errors='coerce')
    df_clean['Time'] = pd.to_datetime(df_clean['Time'], format='%H.%M.%S', errors='coerce').dt.time
    
    # Create datetime column for easier time series analysis
    valid_datetime_mask = df_clean['Date'].notna() & df_clean['Time'].notna()
    df_clean['DateTime'] = pd.NaT
    
    if valid_datetime_mask.any():
        df_clean.loc[valid_datetime_mask, 'DateTime'] = pd.to_datetime(
            df_clean.loc[valid_datetime_mask, 'Date'].astype(str) + ' ' + 
            df_clean.loc[valid_datetime_mask, 'Time'].astype(str)
        )
    
    # Convert numeric columns, handling comma as decimal separator
    numeric_columns = ['CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)', 
                       'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 
                       'PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'RH', 'AH']
    
    for col in numeric_columns:
        if col in df_clean.columns:
            # Replace comma with dot for decimal separator
            df_clean[col] = df_clean[col].astype(str).str.replace(',', '.', regex=False)
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    
    # 🌟 ENHANCEMENT: Replace -200 (missing data indicator) with NaN 🌟
    # This must happen AFTER numeric conversion
    # Count -200 values BEFORE cleaning
    before_count = (df_clean == MISSING_VALUE_CODE).sum()
    total_before = before_count.sum()
    
    if total_before > 0:
        print(f"\n🔍 Found -200 values in {(before_count > 0).sum()} columns:")
        print(before_count[before_count > 0])
    
    # Replace -200 with NaN across all columns
    df_clean = df_clean.replace(MISSING_VALUE_CODE, np.nan)
    
    # Verify AFTER cleaning
    after_count = (df_clean == MISSING_VALUE_CODE).sum().sum()
    print(f"\n✅ Replaced {total_before} instances of -200 with NaN")
    print(f"✅ Remaining -200 values: {after_count}")
    
    return df_clean

In [23]:
clean_data(df)


🔍 Found -200 values in 13 columns:
CO(GT)           1683
PT08.S1(CO)       366
NMHC(GT)         8443
C6H6(GT)          366
PT08.S2(NMHC)     366
NOx(GT)          1639
PT08.S3(NOx)      366
NO2(GT)          1642
PT08.S4(NO2)      366
PT08.S5(O3)       366
T                 366
RH                366
AH                366
dtype: int64

✅ Replaced 16701 instances of -200 with NaN
✅ Remaining -200 values: 0


,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16,DateTime
0,2004-03-10,18:00:00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,NaN,NaN,2004-03-10 18:00:00
1,2004-03-10,19:00:00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,NaN,NaN,2004-03-10 19:00:00
2,2004-03-10,20:00:00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,NaN,NaN,2004-03-10 20:00:00
3,2004-03-10,21:00:00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,NaN,NaN,2004-03-10 21:00:00
4,2004-03-10,22:00:00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,NaN,NaN,2004-03-10 22:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9466,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
9467,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
9468,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
9469,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [24]:
# After cleaning
df_clean = clean_data(df)

# Check for any -200 values
print("Checking for -200 values in cleaned data:")
print((df_clean == -200).sum())
print(f"\nTotal -200 values: {(df_clean == -200).sum().sum()}")

# Also check a few specific values to see what's actually there
print("\nSample of numeric columns:")
print(df_clean[['CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'T', 'RH']].head(20))


🔍 Found -200 values in 13 columns:
CO(GT)           1683
PT08.S1(CO)       366
NMHC(GT)         8443
C6H6(GT)          366
PT08.S2(NMHC)     366
NOx(GT)          1639
PT08.S3(NOx)      366
NO2(GT)          1642
PT08.S4(NO2)      366
PT08.S5(O3)       366
T                 366
RH                366
AH                366
dtype: int64

✅ Replaced 16701 instances of -200 with NaN
✅ Remaining -200 values: 0
Checking for -200 values in cleaned data:
Date             0
Time             0
CO(GT)           0
PT08.S1(CO)      0
NMHC(GT)         0
C6H6(GT)         0
PT08.S2(NMHC)    0
NOx(GT)          0
PT08.S3(NOx)     0
NO2(GT)          0
PT08.S4(NO2)     0
PT08.S5(O3)      0
T                0
RH               0
AH               0
Unnamed: 15      0
Unnamed: 16      0
DateTime         0
dtype: int64

Total -200 values: 0

Sample of numeric columns:
    CO(GT)  PT08.S1(CO)  NMHC(GT)     T    RH
0      2.6       1360.0     150.0  13.6  48.9
1      2.0       1292.0     112.0  13.3  47.7
2      2